In [36]:
import pandas as pd
import requests
from pymongo import MongoClient
from bs4 import BeautifulSoup as BS
import time
import pprint
import json
import requesocks

In [147]:
ls

alameda.csv          mission.csv          potrero_hill.csv     sunset_district.csv
bernal_heights.csv   pacific_heights.csv  south_of_market.csv  twin_peaks.csv


In [38]:
df1 = pd.read_csv('alameda.csv')[['address', 'zid']]
df2 = pd.read_csv('mission.csv')[['address', 'zid']]
df3 = pd.read_csv('potrero_hill.csv')[['address', 'zid']]
df4 = pd.read_csv('sunset_district.csv')[['address', 'zid']]
df5 = pd.read_csv('bernal_heights.csv')[['address', 'zid']]
df6 = pd.read_csv('pacific_heights.csv')[['address', 'zid']]
df7 = pd.read_csv('south_of_market.csv')[['address', 'zid']]
df8 = pd.read_csv('twin_peaks.csv')[['address', 'zid']]

In [148]:
df_sf = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8],
               ignore_index=True,
               axis=0)

In [149]:
df_sf = df_sf.drop_duplicates()

In [39]:
# df1 = pd.read_csv('ballard.csv')[['address', 'zid']]
# df2 = pd.read_csv('beacon_hill.csv')[['address', 'zid']]
# df3 = pd.read_csv('bellevue_v1.csv')[['address', 'zid']]
# df4 = pd.read_csv('bellevue_v2.csv')[['address', 'zid']]
# df5 = pd.read_csv('burien_v2.csv')[['address', 'zid']]
# df6 = pd.read_csv('cap_hill.csv')[['address', 'zid']]
# df7 = pd.read_csv('issaquah_v1.csv')[['address', 'zid']]
# df8 = pd.read_csv('issaquah_v2.csv')[['address', 'zid']]
# df9 = pd.read_csv('lake_union.csv')[['address', 'zid']]
# df10 = pd.read_csv('magnolia.csv')[['address', 'zid']]
# df11 = pd.read_csv('queen_anne.csv')[['address', 'zid']]

In [40]:
# df_sea = pd.concat([df1, df2, df3, df4, df5, df6,
#                     df7, df8, df9, df10, df11],
#                    ignore_index=True,
#                    axis=0)

In [41]:
# df_sea = df_sea.drop_duplicates()

In [42]:
# df_sea.shape

(1350, 2)

In [43]:
# http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=<ZWSID>&zpid=48749425
ZWSID =  "X1-ZWz1a49jy1eadn_7dhgw"
BASE_URL = "http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=" + ZWSID + "&zpid="

In [51]:
def get_property_info(zpid):
    return requests.get(BASE_URL + zpid)

In [52]:
# Get some queries
def query_api(df, start_row, stop_row, proxy):
    queries = []
    df = df.ix[start_row : stop_row]
    for row in df.iterrows():
        zpid = str(row[1]['zid'])
        print row[0]
        queries.append(get_property_info(zpid))
    return queries

In [155]:
queries = query_api(df_sf, 0, 2, False)

0
1
2


In [157]:
queries[0].content

'<?xml version="1.0" encoding="utf-8"?><UpdatedPropertyDetails:updatedPropertyDetails xmlns:UpdatedPropertyDetails="http://www.zillow.com/static/xsd/UpdatedPropertyDetails.xsd" xsi:schemaLocation="http://www.zillow.com/static/xsd/UpdatedPropertyDetails.xsd http://www.zillowstatic.com/vstatic/34794f0/static/xsd/UpdatedPropertyDetails.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><request><zpid>83231047</zpid></request><message><text>Error: this account has reached is maximum number of calls for today</text><code>7</code></message></UpdatedPropertyDetails:updatedPropertyDetails><!-- H:003  T:1ms  S:115  R:Sun Sep 13 22:21:18 PDT 2015  B:4.0.19615-release_20150908-endor.2fa5797~candidate.358c83d -->'

In [136]:
def get_protected_listings(df, queries):
    protected_listings = []
    for query in queries:
        soup = BS(query.content, 'html.parser')
        if soup.find('text').string == 'Error: protected data is unavailable through API':
            zpid = soup.find('zpid').string
            protected_listings.append(int(zpid))
    return df[df['zid'].isin(protected_listings)]

In [151]:
protected_listings = get_protected_listings(df_sf, queries)

In [152]:
protected_listings.shape

(0, 2)

In [141]:
protected_listings.to_csv('sea_protected_listings.csv', separator=',')

In [121]:
scrape_base_url = "http://www.zillow.com/homes/48749340_zpid/?hdpRedirected=true"

In [56]:
def get_listing_tags(soup):
    response_content = soup.find('response').contents
    attributes = []
    for line in response_content:
        if len(line.contents) > 1:
            for sub_line in line.contents:
                attributes.append((line.name, sub_line.name))
        else:
            attributes.append((line.name))
    return attributes  # returns all of the tags and children tags in a list       

In [57]:
def extract_features(tags, soup):
    schema = {}
    for tag in tags:
        if len(tag) == 2:
            if tag[0] not in schema:
                schema[str(tag[0])] = {}
            schema[str(tag[0])][str(tag[1])] = str(soup.find(tag[0]).find(tag[1]).string)
        if len(tag) > 2:
            try:
                schema[str(tag)] = str(soup.find(tag).string)
            except:
                continue
    return schema # returns a dict with all of the info from the listing, formatted for insertion in to MongoDB

In [58]:
# Now turn all of the html content into a json like structure
# for insertion into MongoDB

def structure_data(queries):
    entries = []
    for query in queries: # a query is just a http request response object
            content = query.content
            soup = BS(content, 'html.parser')
            code = int(soup.find('code').string)
            if code == 0:  # a good api call
                tags = get_listing_tags(soup)
                entry = extract_features(tags, soup)
                entries.append(entry)
    return entries

In [153]:
entries = structure_data(queries)

In [154]:
len(entries)

0

In [66]:
# with open('entries_2.json') as f:
#     entries_2 = f.readlines()

In [79]:
entries2 = entries

In [88]:
def insert_queries(entries):
    client = MongoClient()
    db = client['proj']
    listings = db['listings']
    for entry in entries:
        listings.insert(entry)
        #print result.inserted_id

In [90]:
insert_queries(entries)

In [278]:
# write these things to file before I lose them!!!

# with open('entries_2.json', 'w') as j:
#     for line in entries_2:
#         j.write(str(line) + '\n')
#     j.close()

In [53]:
#entries